In [1]:

import PySimpleGUI as sg
import requests
from time import sleep
import pandas as pd
import unicodedata
import string
import urllib
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#③実行する
import gspread
import json
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials 
from gspread_dataframe import get_as_dataframe, set_with_dataframe
#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
import ssl
from datetime import datetime as dt
ssl._create_default_https_context = ssl._create_unverified_context
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
# from utils import *
import datetime
import time
import os

def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 


def read_spreadsheet(workbook,worksheet):
    read_worksheet_df = pd.DataFrame(worksheet.get_all_values())
    read_worksheet_df.columns = list(read_worksheet_df.loc[0, :])
    read_worksheet_df.drop(0, inplace=True)
    return read_worksheet_df

def write_dataframe_to_spreadsheet(write_spreadsheet_df,worksheet:str,title_text:str):
    workbook.add_worksheet(title=title_text, rows=len(write_spreadsheet_df), cols=len(write_spreadsheet_df.columns))
    worksheet = workbook.worksheet(title_text)
    #df_a['G数'] = df_a['G数'].map(set_round)'
    set_with_dataframe(worksheet, write_spreadsheet_df)

In [ ]:
#備忘録
##

#1,サミーの日のSFデータをダウンロードする


#サミーの日取得すべき店舗数を把握する


#2,サミーの日の店舗名をグーグルで検索してアナスロの店舗名で辞書型で取得する

In [6]:
sammy_sf_report_df = pd.read_excel(r'csv\hollnabi_syoten_0502.xlsx')
sammy_sf_report_df
sammy_sf_report_df = sammy_sf_report_df[sammy_sf_report_df['イベント日'] <= pd.Timestamp(2023,5,31)]
sammy_sf_report_df

,イベント日,店舗名／法人名,都道府県,提案区分,ドリスロ提案区分,初開催フラグ,合同企画提案区分
0,2023-05-01,ピーアーク朝霞,埼玉県,昇天チャレンジ,NaN,NaN,NaN
1,2023-05-03,グランキコーナ泉佐野店,大阪府,昇天チャレンジ,NaN,NaN,NaN
2,2023-05-03,ピーアーク越谷店,埼玉県,昇天チャレンジ,NaN,NaN,NaN
3,2023-05-03,アビバ湘南台店,神奈川県,昇天チャレンジ,NaN,NaN,NaN
4,2023-05-03,ピーアーク松原店,埼玉県,昇天チャレンジ,NaN,NaN,NaN
...,...,...,...,...,...,...,...
169,2023-05-29,グランキコーナ泉佐野店,大阪府,昇天チャレンジ,NaN,NaN,NaN
170,2023-05-29,楽園上新屋店,静岡県,昇天チャレンジ,NaN,NaN,NaN
171,2023-05-30,楽園上新屋店,静岡県,昇天チャレンジ,NaN,NaN,NaN
172,2023-05-31,FACE1100瑞穂,埼玉県,昇天チャレンジ,NaN,NaN,NaN


In [7]:
len(sammy_sf_report_df)

174

In [8]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

anaslo_scraping_error_tenpo_name_list = []
browser = webdriver.Chrome(ChromeDriverManager().install())
serched_df = pd.DataFrame(index=[], columns=[])
slopachi_and_anaslo_tenpo_name_dict = {}
#no_scraping_anaslo_tenpo_name = []
for slopachi_tenpo_name in sorted(list(sammy_sf_report_df['店舗名／法人名'].unique())):
    url = f"https://www.google.com/search?q=アナスロ  {slopachi_tenpo_name} データ一覧&hl=ja&source=hp/"
    print(url)
    #print('データ取得中...30秒ほど時間がかかります。')
    browser.get(url)
    html = browser.page_source.encode('utf-8')
    elems_rec = browser.find_elements_by_class_name('MjjYud')
    elems_rec[0].text.split('\n')
    print('アナスロと出る部分',elems_rec[0].text.split('\n')[1])

    scraping_anaslo_name = elems_rec[0].text.split('\n')[0]
    slopachi_and_anaslo_tenpo_name_dict[slopachi_tenpo_name] = scraping_anaslo_name

    time.sleep(2)
    # except Exception as e:
    #     anaslo_scraping_error_tenpo_name_list.append(tenpo_name)
    #     print('エラー店舗',tenpo_name,e)
    #     break
#data_df.to_csv(csv_file_name)



====== WebDriver manager ======
Current google-chrome version is 113.0.5672
Get LATEST chromedriver version for 113.0.5672 google-chrome
Driver [C:\Users\slotd\.wdm\drivers\chromedriver\win32\113.0.5672.63\chromedriver.exe] found in cache
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/556597382.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


https://www.google.com/search?q=アナスロ  123+N和歌山本店 データ一覧&hl=ja&source=hp/


C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/556597382.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elems_rec = browser.find_elements_by_class_name('MjjYud')


アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  123+N大阪本店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  123+N松原 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  123+N松原店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  123門真店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  D'station上田店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  FACE1100瑞穂 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  MGM小山本郷店 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  せんげん台DUO データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  みずほ台UNO データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  アビバアネックス データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  アビバ三浦海岸 データ一覧&hl=ja&source=hp/
アナスロと出る部分 アナスロ
https://www.google.com/search?q=アナスロ  アビバ上大岡 データ

In [15]:
##出せない店舗名一覧
convert_tenponame_dict_df[convert_tenponame_dict_df['変換後'].str.contains('データ一覧')]

,変換前,変換後


In [9]:
convert_tenponame_dict_df = pd.DataFrame(slopachi_and_anaslo_tenpo_name_dict.values(), index=slopachi_and_anaslo_tenpo_name_dict.keys())
convert_tenponame_dict_df = convert_tenponame_dict_df.reset_index()
convert_tenponame_dict_df  = convert_tenponame_dict_df.rename(columns={'index': '変換前',0: '変換後'})
convert_tenponame_dict_df = convert_tenponame_dict_df[convert_tenponame_dict_df['変換後'].str.contains('データ一覧')]
convert_tenponame_dict_df['変換後'] = convert_tenponame_dict_df['変換後'].apply(lambda x: x.replace(' データ一覧',''))
convert_tenponame_dict_df.to_csv(r'csv\dict_0602_昇天チャレンジ_アナスロ店舗名変換データ一覧.csv')
convert_tenponame_dict_df
#データ一覧のみを含む店舗名法人名とアナスロ店舗名の辞書型
extraxt_convert_tenponame_dict = dict(zip(convert_tenponame_dict_df['変換前'], convert_tenponame_dict_df['変換後']))
extraxt_convert_tenponame_dict

{'123+N和歌山本店': '123+N和歌山本店',
 '123+N大阪本店': '123+N大阪本店',
 '123+N松原': '123+N松原店',
 '123+N松原店': '123+N松原店',
 '123門真店': '123門真店',
 "D'station上田店": 'Dステーション上田店',
 'FACE1100瑞穂': 'フェイス1100瑞穂店',
 'MGM小山本郷店': 'MGM小山本郷店',
 'せんげん台DUO': 'せんげん台DUO',
 'みずほ台UNO': 'みずほ台UNO',
 'アビバアネックス': 'アビバアネックス・スクエア店',
 'アビバ三浦海岸': 'アビバ三浦海岸店',
 'アビバ上大岡': 'アビバ上大岡店',
 'アビバ南足柄': 'アビバ南足柄店',
 'アビバ南足柄店': 'アビバ南足柄店',
 'アビバ新杉田': 'アビバ新杉田',
 'アビバ新杉田店': 'アビバ新杉田',
 'アビバ横須賀中央': 'アビバ横須賀中央店',
 'アビバ湘南台店': 'アビバ湘南台店',
 'アビバ綱島樽町': 'アビバ綱島樽町店',
 'アビバ逗子駅前': 'アビバ逗子駅前店',
 'アビバ逗子駅前店': 'アビバ逗子駅前店',
 'アビバ関内': 'アビバ関内',
 'アビバ鶴見': 'アビバ鶴見店',
 'キコーナ小田原扇町': 'キコーナ小田原扇町店',
 'キコーナ小田原扇町店': 'キコーナ小田原扇町店',
 'グランキコーナ泉佐野': 'グランキコーナ泉佐野店',
 'グランキコーナ泉佐野店': 'グランキコーナ泉佐野店',
 'グランキコーナ相模原': 'グランキコーナ相模原店',
 'グランキコーナ相模原店': 'グランキコーナ相模原店',
 'ジャムフレンドクラブ朝霞店': 'ジャムフレンドクラブ朝霞店',
 'テンガイ与次郎店': 'テンガイ・与次郎店',
 'パラッツォ三郷中央': 'パラッツォ三郷中央店',
 'パラッツォ川越': 'パラッツォ川越店',
 'パラッツォ秋津': 'パラッツォ秋津店',
 'パラッツォ鳩ヶ谷': 'パラッツォ鳩ヶ谷店',
 'パールショップともえ多古店': 'パールショップともえ多古店',
 'パールショップともえ大和560': 'パールショップともえ大和560'

In [10]:
prefecture_name_list = '''北海道
青森県
岩手県
宮城県
秋田県
山形県
福島県
茨城県
栃木県
群馬県
埼玉県
千葉県
東京都
神奈川県
新潟県
富山県
石川県
福井県
山梨県
長野県
岐阜県
静岡県
愛知県
三重県
滋賀県
京都府
大阪府
兵庫県
奈良県
和歌山県
鳥取県
島根県
岡山県
広島県
山口県
徳島県
香川県
愛媛県
高知県
福岡県
佐賀県
長崎県
熊本県
大分県
宮崎県
鹿児島県
沖縄県'''.split('\n')

In [27]:
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import datetime
import time
import unicodedata
import string
import urllib
def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text
def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload)
def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files)
def TablePlot(df,outputPath,w,h,fontsize,title):
    plt.rcParams["font.size"] = fontsize
    plt.rcParams["font.weight"] = "bold"
    fig, ax = plt.subplots(figsize=(w,h))
    ax.axis('off')
    ax.table(cellText=df.values,
             colLabels=df.columns,
             loc='center',
             bbox=[0,0,1,1])
    ax.set_title(f'{title}',
             fontweight ="bold")
    plt.savefig(outputPath)


#print(tenpo_ichiran_df['ホール名'])
cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']
ichiran_all_tennpo_df = pd.DataFrame(index=[], columns=cols)
tenpo_url_df = pd.DataFrame(index=[], columns=cols)
yesterday = datetime.date.today() + datetime.timedelta(days=-1)
browser = webdriver.Chrome(ChromeDriverManager().install())
parlar_name_parlar_url_dict = {}
for prefecture_name in prefecture_name_list:
    url = f'https://ana-slo.com/%E3%83%9B%E3%83%BC%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF/{prefecture_name}/'
    browser.get(url)
    html = browser.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'lxml')
    for element in soup.find(class_="dataTable no-footer").find_all("a"):
        #print(element.string,element['href'])
        parlar_name = element.string
        parlar_name_url = element['href']
        parlar_name_parlar_url_dict[parlar_name] = parlar_name_url
        tenpo_url_df = pd.concat([tenpo_url_df,pd.DataFrame([[prefecture_name,parlar_name,parlar_name_url]])],ignore_index=True)

        



====== WebDriver manager ======
Current google-chrome version is 113.0.5672
Get LATEST chromedriver version for 113.0.5672 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\slotd\.wdm\drivers\chromedriver\win32\113.0.5672.63]
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/1415024754.py:51: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [31]:
tenpo_name_tenpo_url_df = pd.read_csv(r'csv\tenpo_url.csv' )
tenpo_name_tenpo_url_df['url_店舗名'] = tenpo_name_tenpo_url_df['URL'].map(lambda x: urllib.parse.unquote((x.split('/')[-2])).replace('-データ一覧',''))
tenpo_name_tenpo_url_df.to_csv(r'csv\tenpo_url_flask_web_site.csv',index=False)

In [12]:
import csv
dct = parlar_name_parlar_url_dict
with open(r'csv\tenpo_name_tenpo_url_dict.csv', 'w') as f:  
    writer = csv.writer(f)
    for k, v in dct.items():
       writer.writerow([k, v])

tenpo_name_tenpo_url_df = pd.read_csv(r'csv\tenpo_name_tenpo_url_dict.csv', header=None, names=['店舗名','URL'])
tenpo_name_tenpo_url_df['url_店舗名'] = tenpo_name_tenpo_url_df['URL'].map(lambda x: urllib.parse.unquote((x.split('/')[-2])).replace('-データ一覧',''))
tenpo_name_tenpo_url_df.to_csv(r'csv\アナスロ店舗名デコードテーブル.csv',index=False)

In [13]:
tenpo_name_tenpo_url_df

,店舗名,URL,url_店舗名
0,BIG大王東店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,big大王東店
1,CREV伊達店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,crev伊達店
2,EBH,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,ebh
3,INDY函館,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,indy函館
4,KEIO湯川店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,keio湯川店
...,...,...,...
3180,マルシンV3,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,マルシンv3
3181,マルシンV4,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,マルシンv4
3182,ラッキー高原店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,ラッキー高原店
3183,ルクソール,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,ルクソール


In [16]:
sammy_sf_report_df['アナスロ店舗名'] = sammy_sf_report_df['店舗名／法人名'].map(extraxt_convert_tenponame_dict)
sammy_sf_report_df

,イベント日,店舗名／法人名,都道府県,提案区分,ドリスロ提案区分,初開催フラグ,合同企画提案区分,アナスロ店舗名
0,2023-05-01,ピーアーク朝霞,埼玉県,昇天チャレンジ,NaN,NaN,NaN,ピーアーク朝霞
1,2023-05-03,グランキコーナ泉佐野店,大阪府,昇天チャレンジ,NaN,NaN,NaN,グランキコーナ泉佐野店
2,2023-05-03,ピーアーク越谷店,埼玉県,昇天チャレンジ,NaN,NaN,NaN,ピーアーク越谷
3,2023-05-03,アビバ湘南台店,神奈川県,昇天チャレンジ,NaN,NaN,NaN,アビバ湘南台店
4,2023-05-03,ピーアーク松原店,埼玉県,昇天チャレンジ,NaN,NaN,NaN,ピーアーク松原
...,...,...,...,...,...,...,...,...
169,2023-05-29,グランキコーナ泉佐野店,大阪府,昇天チャレンジ,NaN,NaN,NaN,グランキコーナ泉佐野店
170,2023-05-29,楽園上新屋店,静岡県,昇天チャレンジ,NaN,NaN,NaN,楽園上新屋店
171,2023-05-30,楽園上新屋店,静岡県,昇天チャレンジ,NaN,NaN,NaN,楽園上新屋店
172,2023-05-31,FACE1100瑞穂,埼玉県,昇天チャレンジ,NaN,NaN,NaN,フェイス1100瑞穂店


In [17]:
anaslo_scraping_error_tenpo_name_list = []
browser = webdriver.Chrome(ChromeDriverManager().install())
serched_df = pd.DataFrame(index=[], columns=[])
# for target_day in list(sammy_sf_report_df['イベント日'].unique()):
#     print(target_day)
error_tenpo_name_list = []
anaslo_scraping_error_tenpo_name_list  = []
for i, row in sammy_sf_report_df.iterrows():
    try:
        tenpo_name = row['アナスロ店舗名']
        date = row['イベント日'].to_pydatetime().strftime('%Y-%m-%d')
        anaslo_parlar_name_url = parlar_name_parlar_url_dict[tenpo_name]
        anaslo_url_parlarname =urllib.parse.unquote((anaslo_parlar_name_url.split('/')[-2])).replace('-データ一覧','')
        url = f"https://ana-slo.com/{date}-{anaslo_url_parlarname}-data/"
        print(url)
        #print('データ取得中...30秒ほど時間がかかります。')
        browser.get(url)
        html = browser.page_source.encode('utf-8')
        dfs = pd.read_html(html)
        #display(tenpo_df)
        #time.sleep(1)
        elements = browser.find_element(By.CLASS_NAME, 'st-catgroup')
        prefecture = elements.text.split(' ')[-1]
        for df in  dfs:
        #print(df.columns)
            if '機種名' in list(df.columns):
                ichiran_df = df
                ichiran_df['店舗名'] = tenpo_name
                #ichiran_df['都道府県'] = prefecture
                ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                first_column = ichiran_df.pop('店舗名')
                ichiran_df.insert(0,'店舗名',first_column)
                break
        tenpo_day_data_df = ichiran_df
        tenpo_day_data_df = tenpo_day_data_df.sort_values('台番号')
        tenpo_day_data_df['日付'] = f"{date}"
        serched_df = pd.concat([serched_df,tenpo_day_data_df],axis=0)
    except Exception as e:
        anaslo_scraping_error_tenpo_name_list.append(tenpo_name)
        print('エラー店舗',tenpo_name,e)
        continue
    # if i > 2:
    #     break




====== WebDriver manager ======
Current google-chrome version is 113.0.5672
Get LATEST chromedriver version for 113.0.5672 google-chrome
Driver [C:\Users\slotd\.wdm\drivers\chromedriver\win32\113.0.5672.63\chromedriver.exe] found in cache
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/222010996.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


https://ana-slo.com/2023-05-01-ピーアーク朝霞-data/
https://ana-slo.com/2023-05-03-グランキコーナ泉佐野店-data/
エラー店舗 グランキコーナ泉佐野店 No tables found
https://ana-slo.com/2023-05-03-ピーアーク越谷-data/
https://ana-slo.com/2023-05-03-アビバ湘南台店-data/
https://ana-slo.com/2023-05-03-ピーアーク松原-data/
https://ana-slo.com/2023-05-03-キコーナ小田原扇町店-data/
https://ana-slo.com/2023-05-05-グランキコーナ泉佐野店-data/
エラー店舗 グランキコーナ泉佐野店 No tables found
https://ana-slo.com/2023-05-05-ピーアーク相模原-data/
https://ana-slo.com/2023-05-06-中山uno-data/
https://ana-slo.com/2023-05-06-中山zoron-data/
https://ana-slo.com/2023-05-06-鶴見uno-data/
https://ana-slo.com/2023-05-06-上福岡uno-data/
https://ana-slo.com/2023-05-06-北越谷uno-data/
https://ana-slo.com/2023-05-06-せんげん台duo-data/
https://ana-slo.com/2023-05-06-みずほ台uno-data/
https://ana-slo.com/2023-05-06-本八幡uno-data/
https://ana-slo.com/2023-05-06-南柏uno-data/
https://ana-slo.com/2023-05-06-稲毛海岸uno-data/
https://ana-slo.com/2023-05-06-パラッツォ三郷中央店-data/
https://ana-slo.com/2023-05-06-アビバ上大岡店-data/
https://ana-slo.com/2023-

In [53]:
hokuto_average_out = int(extraxt_hokuto_kisyu_name_df.mean()['G数'] * 3)
print(hokuto_average_out)
hokuto_average_samai = int(extraxt_hokuto_kisyu_name_df.mean()['差枚'] )
print(hokuto_average_samai)
hokuto_average_daisuu = len(extraxt_hokuto_kisyu_name_df)
print(hokuto_average_daisuu)

extraxt_tenpo_average_out = int(extract_tenpo_data_df.mean()['G数'] * 3)
print(extraxt_tenpo_average_out)
extraxt_tenpo_samai = int(extract_tenpo_data_df.mean()['差枚'] )
print(extraxt_tenpo_samai)
extraxt_tenpo_daisuu = len(extract_tenpo_data_df)
print(extraxt_tenpo_daisuu)

20774
-344
9
7642
-225
150


C:\Users\slotd\AppData\Local\Temp/ipykernel_13048/408010251.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_out = int(extraxt_hokuto_kisyu_name_df.mean()['G数'] * 3)
C:\Users\slotd\AppData\Local\Temp/ipykernel_13048/408010251.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_samai = int(extraxt_hokuto_kisyu_name_df.mean()['差枚'] )
C:\Users\slotd\AppData\Local\Temp/ipykernel_13048/408010251.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  extraxt_ten

In [21]:
error_tenpo_name_dict = {}
output_df = pd.DataFrame(index=[],columns= \
    ['イベント日','店舗名／法人名','アナスロ店舗名','都道府県','提案区分',\
    'スマスロ北斗平均アウト','スマスロ北斗平均差枚','スマスロ北斗台数',\
    '全体平均アウト','全体平均差枚','全体台数'])

for i, row in sammy_sf_report_df.iterrows():
    try:
        date = row['イベント日'].to_pydatetime().strftime('%Y-%m-%d')
        serch_tenpo_name =row['アナスロ店舗名']
        extract_tenpo_data_df = serched_df[(serched_df['店舗名'] == serch_tenpo_name) & (serched_df['日付'] == date)] 
        if len(extract_tenpo_data_df) == 0:
            #print(row)
            hokuto_average_out = hokuto_average_samai = hokuto_average_daisuu = \
            extraxt_tenpo_average_out = extraxt_tenpo_samai  = extraxt_tenpo_daisuu = 'データなし'
            error_tenpo_name_dict[serch_tenpo_name] = 'データテーブルがないです'
            
        else:
            display(extract_tenpo_data_df)
            extraxt_hokuto_kisyu_name_df = extract_tenpo_data_df[extract_tenpo_data_df['機種名'].str.contains('北斗の拳') ]
            extraxt_hokuto_kisyu_name_df = extraxt_hokuto_kisyu_name_df[~extraxt_hokuto_kisyu_name_df['機種名'].str.contains('宿命') ]
            extraxt_hokuto_kisyu_name_df = extraxt_hokuto_kisyu_name_df[~extraxt_hokuto_kisyu_name_df['機種名'].str.contains('天昇') ]
            extraxt_hokuto_kisyu_name_df
            hokuto_average_out = int(extraxt_hokuto_kisyu_name_df.mean()['G数'] * 3)
            print(hokuto_average_out)
            hokuto_average_samai = int(extraxt_hokuto_kisyu_name_df.mean()['差枚'] )
            print(hokuto_average_samai)
            hokuto_average_daisuu = len(extraxt_hokuto_kisyu_name_df)
            print(hokuto_average_daisuu)

            extraxt_tenpo_average_out = int(extract_tenpo_data_df.mean()['G数'] * 3)
            print(extraxt_tenpo_average_out)
            extraxt_tenpo_samai = int(extract_tenpo_data_df.mean()['差枚'] )
            print(extraxt_tenpo_samai)
            extraxt_tenpo_daisuu = len(extract_tenpo_data_df)
            print(extraxt_tenpo_daisuu)


        record = [[date, row['店舗名／法人名'], row['アナスロ店舗名'], row['都道府県'],row['提案区分'],\
                hokuto_average_out,hokuto_average_samai,hokuto_average_daisuu,\
                extraxt_tenpo_average_out,extraxt_tenpo_samai ,extraxt_tenpo_daisuu]]
        df_append = pd.DataFrame(data=record, columns=output_df.columns)
        output_df = pd.concat([output_df, df_append], ignore_index=True, axis=0)
    #break
    except Exception as e:
        error_tenpo_name_dict[serch_tenpo_name] = e
        print('エラー店舗',serch_tenpo_name,e)
        continue
        

,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
52,ピーアーク朝霞,HEYエリートサラリーマン鏡,234,5644,-2193.0,23,9,0.0,1/176.4,1/245.4,1/627.1,1/0.0,2023-05-01
53,ピーアーク朝霞,HEYエリートサラリーマン鏡,235,1334,210.0,7,4,0.0,1/121.3,1/190.6,1/333.5,1/0.0,2023-05-01
54,ピーアーク朝霞,HEYエリートサラリーマン鏡,236,3385,562.0,21,8,0.0,1/116.7,1/161.2,1/423.1,1/0.0,2023-05-01
55,ピーアーク朝霞,HEYエリートサラリーマン鏡,237,839,-110.0,3,1,0.0,1/209.8,1/279.7,1/839.0,1/0.0,2023-05-01
56,ピーアーク朝霞,HEYエリートサラリーマン鏡,238,1601,-1252.0,7,1,0.0,1/200.1,1/228.7,1/1601.0,1/0.0,2023-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,ピーアーク朝霞,マイジャグラーV,379,1312,-867.0,2,1,0.0,1/437.3,1/656.0,1/1312.0,1/0.0,2023-05-01
19,ピーアーク朝霞,マイジャグラーV,380,1349,-267.0,3,6,0.0,1/149.9,1/449.7,1/224.8,1/0.0,2023-05-01
20,ピーアーク朝霞,マイジャグラーV,381,6383,-1839.0,17,19,0.0,1/177.3,1/375.5,1/335.9,1/0.0,2023-05-01
21,ピーアーク朝霞,マイジャグラーV,382,1574,-288.0,6,1,0.0,1/224.9,1/262.3,1/1574.0,1/0.0,2023-05-01


20774
-344
9
7642
-225
150


C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_out = int(extraxt_hokuto_kisyu_name_df.mean()['G数'] * 3)
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_samai = int(extraxt_hokuto_kisyu_name_df.mean()['差枚'] )
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:31: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  extra

,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
200,ピーアーク越谷,沖ドキDUO30,501,305,-513.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-03
201,ピーアーク越谷,沖ドキDUO30,502,2688,-1388.0,13,2,0.0,1/179.2,1/206.8,1/1344.0,1/0.0,2023-05-03
202,ピーアーク越谷,沖ドキDUO30,503,1189,135.0,9,2,0.0,1/108.1,1/132.1,1/594.5,1/0.0,2023-05-03
203,ピーアーク越谷,沖ドキDUO30,504,297,1112.0,7,1,0.0,1/37.1,1/42.4,1/297.0,1/0.0,2023-05-03
92,ピーアーク越谷,沖ドキGOLD30,505,2453,-388.0,15,5,0.0,1/122.7,1/163.5,1/490.6,1/0.0,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,ピーアーク越谷,幼女戦記,792,2055,-290.0,0,2,47.0,1/41.9,1/0.0,1/1027.5,1/43.7,2023-05-03
256,ピーアーク越谷,鉄拳5,793,2788,-1170.0,10,4,14.0,1/99.6,1/278.8,1/697.0,1/199.1,2023-05-03
257,ピーアーク越谷,FAIRYTAIL2,794,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-03
258,ピーアーク越谷,黄門ちゃま喝2,795,957,-742.0,0,0,6.0,1/159.5,1/0.0,1/0.0,1/159.5,2023-05-03


16617
314
12
8604
-210
260


...

14342
57
34
5584
-31
539


C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_out = int(extraxt_hokuto_kisyu_name_df.mean()['G数'] * 3)
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_samai = int(extraxt_hokuto_kisyu_name_df.mean()['差枚'] )
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:31: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  extra

,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
230,楽園上新屋店,ファンキージャグラー2,101,3875,-285.0,12,17,0.0,1/133.6,1/322.9,1/227.9,1/0.0,2023-05-31
231,楽園上新屋店,ファンキージャグラー2,102,3173,1381.0,18,8,0.0,1/122.0,1/176.3,1/396.6,1/0.0,2023-05-31
232,楽園上新屋店,ファンキージャグラー2,103,3042,-1381.0,7,7,0.0,1/217.3,1/434.6,1/434.6,1/0.0,2023-05-31
233,楽園上新屋店,ファンキージャグラー2,104,1234,-48.0,5,2,0.0,1/176.3,1/246.8,1/617.0,1/0.0,2023-05-31
234,楽園上新屋店,ファンキージャグラー2,105,1730,-285.0,6,5,0.0,1/157.3,1/288.3,1/346.0,1/0.0,2023-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,楽園上新屋店,HEYエリートサラリーマン鏡,3405,441,-476.0,0,0,1.0,1/441.0,1/0.0,1/0.0,1/441.0,2023-05-31
227,楽園上新屋店,HEYエリートサラリーマン鏡,3406,3893,96.0,0,9,22.0,1/125.6,1/0.0,1/432.6,1/177.0,2023-05-31
228,楽園上新屋店,HEYエリートサラリーマン鏡,3407,11,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-31
229,楽園上新屋店,HEYエリートサラリーマン鏡,3408,1114,2096.0,0,7,9.0,1/69.6,1/0.0,1/159.1,1/123.8,2023-05-31


14826
38
34
6141
-33
539


C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_out = int(extraxt_hokuto_kisyu_name_df.mean()['G数'] * 3)
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:26: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  hokuto_average_samai = int(extraxt_hokuto_kisyu_name_df.mean()['差枚'] )
C:\Users\slotd\AppData\Local\Temp/ipykernel_33456/2012436428.py:31: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  extra

In [19]:

output_df.to_csv('csv/5月分昇天チャレンジ結果.csv', index=False)

In [26]:
error_tenpo_name_dict

{'グランキコーナ泉佐野店': 'データテーブルがないです',
 'ピーアーク相模原': ValueError('cannot convert float NaN to integer'),
 '123門真店': ValueError('cannot convert float NaN to integer'),
 nan: 'データテーブルがないです',
 'ワンダーランド大刀洗店': 'データテーブルがないです',
 'Dステーション上田店': 'データテーブルがないです',
 'ワンダーランド770東合川店': 'データテーブルがないです',
 'ワンダーランド香椎本館': 'データテーブルがないです',
 'ワンダーランド南ヶ丘店': 'データテーブルがないです',
 'ピーアーク相模大野': ValueError('cannot convert float NaN to integer'),
 'ワンダーランド鹿児島新栄店': 'データテーブルがないです',
 'ワンダーランド谷山店': 'データテーブルがないです',
 'フェイス1100瑞穂店': 'データテーブルがないです',
 'ワンダーランド諫早店': 'データテーブルがないです',
 'ワンダーランド西新店': 'データテーブルがないです',
 'ワンダーランド香椎II': 'データテーブルがないです'}

In [ ]:
error_kisyu_name_list:list[str] = []
error_tenpo_name_list:list[str] = []
sammy_groupby_serched_df_list = []
for i, row in sammy_sf_report_df.iterrows():
    target_date = row['日付']

    extract_tenpo_data_df = data_df[data_df['店舗名'] == serch_tenpo_name]
    print(serch_tenpo_name,len(extract_tenpo_data_df))
    for kisyu_name in serch_kisyu_name_list :
        #print(kisyu_name)
        try:
            serch_kisyu_name_record = extract_tenpo_data_df[extract_tenpo_data_df['機種名'] == kisyu_name]
            if len(serch_kisyu_name_record) == 0:
                error_kisyu_name_list.append(kisyu_name)# + f'_{date_str}'
            serched_df= pd.concat([serched_df, serch_kisyu_name_record], axis=0)
            #print('成功',kisyu_name)
        except:
            print('失敗',kisyu_name)
            error_kisyu_name_list.append(kisyu_name)
            continue
        #break
    # except:
    #     error_tenpo_name_list.append(serch_tenpo_name)
    #     continue
    #display(extract_tenpo_data_df)
    #break
    #break
#serch_kisyu_name_list.remove('')
serched_df['G数'] = serched_df['G数'].astype(int)
groupby_serched_df = serched_df.groupby('店舗名').mean()#.sort_values('G数',ascending=False)#.reset_index()
groupby_serched_df['サミー総台数'] = serched_df.groupby(['店舗名']).size()
new_column_gamesuu_column:str = target_date + '_' + 'サミー全台平均G数'
groupby_serched_df = groupby_serched_df.rename(columns={'G数':new_column_gamesuu_column})
groupby_serched_df = groupby_serched_df[[ new_column_gamesuu_column,'サミー総台数']]#groupby_serched_df_list
sammy_groupby_serched_df_list.append(groupby_serched_df)


In [25]:
date_list = ['2023-05-02','2023-05-03']
for date_str in date_list:
    serched_df[serched_df['日付'] == date_str].to_csv(fr'csv\2023-{date_str}_scraping_anaslo.csv')

In [26]:
concat_data_df = pd.DataFrame(index=[], columns=[])

for date_str in date_list:
    csv_file_name  = f'csv\\2023-{date_str}_scraping_anaslo.csv'
    data_df =  pd.read_csv(csv_file_name)
    concat_data_df=  pd.concat([concat_data_df, data_df], axis=0)



In [29]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
#認証情報設定
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'sercret_json\twitteranalytics-jsonsercretkey.json', scope)
gc = gspread.authorize(credentials)
workbook = gc.open_by_key('105cZcOMvRnx4hNA-N7oypGwIXuetCRZJEe1IHf9R45c')
worksheet = workbook.worksheet('機種名変換テーブル')
kisyu_name_convert_df = read_spreadsheet(workbook,worksheet)
kisyu_name_convert_df

serch_kisyu_name_list:list[str] = list(kisyu_name_convert_df['変換前アナスロ機種名'])
serch_kisyu_name_list = list(set(serch_kisyu_name_list))
serch_kisyu_name_list = sorted(serch_kisyu_name_list)
#serch_kisyu_name_list.remove('')
print(len(serch_kisyu_name_list ))
serch_kisyu_name_list.remove('')
serch_kisyu_name_list 

75


['43Lパチスロ北斗の拳ADXR',
 'ANEMONE 交響詩篇エウレカセブン HI‐EVOLUTION',
 'ANEMONE交響詩篇エウレカセブンHIEVOLUTION',
 'Angel Beats!',
 'AngelBeats',
 'Lスマスロ北斗',
 'Lパチスロ北斗の拳ADXR',
 'RYUKYUBEAT30',
 'S Angel beats! XF',
 'S コードギアス',
 'S パチスロコードギアス',
 'Sエウレカセブン HIEVO ZR',
 'SエウレカセブンHIEVOZR',
 'Sコードギアス反逆のルルーシュ',
 'Sコードギアス叛逆のルルーシュKZ',
 'Sハ-ドボイルドXX',
 'Sパチスロ交響詩篇エウレカセブンHI-EVOLZR',
 'Sパチスロ北斗の拳',
 'Sパチスロ幼女戦姫記ZR',
 'Sヱウレカセブン3',
 'S交響詩篇エウレカセブンHI-EVOL',
 'S北斗の拳昇天',
 '‐S盾勇者成り上がり',
 'あの日見た花の名前を僕達はまだ知らない。',
 'けものフレFS',
 'けものフレンズ',
 'この素晴らしい世界に祝福を',
 'この素晴らしい世界に祝福を！',
 'アラジンAクラシック',
 'エウレカセブン',
 'ガメラ',
 'コードギアス反逆のルルーシュ',
 'コードギアス反逆のルルーシュ3',
 'コードギアス反逆のルルーシュZF',
 'スマスロ北斗の拳',
 'チェインクロニクル',
 'ツインエンジェルP',
 'ツインエンジェルPARTY',
 'ディスクUP2',
 'ディスクアUP2',
 'ディスクアップ2',
 'ドリフターズ',
 'ハードボイXX',
 'ハードボイルド',
 'パチスロ ツインエンジェルBREAK',
 'パチスロディスクアップ',
 'パチスロファンシースターオンライン2',
 'パチスロ交響詩篇エウレカセブ',
 'パチスロ猛獣王 王者の咆',
 'パチスロ猛獣王 王者の咆ZZ',
 'ファンタシースターオンライン2',
 'ペルソナ5',
 '七つの大罪',
 '交響詩篇エウレカセブン3 HI-EVOLUTION ZERO',
 '交響詩篇エウレカセブン3HIEVOLUTIONZERO',
 '北斗の拳 天昇',
 '北斗

In [27]:
concat_data_df

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
0,332,123+N和歌山本店,モンキーターンIV,705,1666,-528.0,0,3,7.0,1/166.6,1/0.0,1/555.3,1/238.0,2023-05-02
1,333,123+N和歌山本店,モンキーターンIV,706,1052,-1003.0,0,2,2.0,1/263.0,1/0.0,1/526.0,1/526.0,2023-05-02
2,334,123+N和歌山本店,モンキーターンIV,707,998,-818.0,0,1,2.0,1/332.7,1/0.0,1/998.0,1/499.0,2023-05-02
3,368,123+N和歌山本店,ゼーガペイン2,708,3,-5.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-02
4,366,123+N和歌山本店,笑ゥせぇるすまん4,709,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28717,131,BEAM by HIKARI,沖ドキGOLD,1172,2214,-1890.0,6,3,NaN,1/246.0,1/369.0,1/738.0,NaN,2023-05-03
28718,132,BEAM by HIKARI,沖ドキGOLD,1173,3163,-2752.0,8,6,NaN,1/225.9,1/395.4,1/527.2,NaN,2023-05-03
28719,133,BEAM by HIKARI,沖ドキGOLD,1175,3198,-1312.0,16,4,NaN,1/159.9,1/199.9,1/799.5,NaN,2023-05-03
28720,134,BEAM by HIKARI,沖ドキGOLD,1176,4409,4833.0,47,20,NaN,1/65.8,1/93.8,1/220.4,NaN,2023-05-03


In [30]:
new_kisyu_name_candidate_list = sorted(list(concat_data_df['機種名'].unique()))
print(len(new_kisyu_name_candidate_list))
kisyu_name_candidate_list = list(set(new_kisyu_name_candidate_list) - set(serch_kisyu_name_list)) 
print(len(kisyu_name_candidate_list))
for x in sorted(kisyu_name_candidate_list):
    print(x)

217
187
2027DB
BIG島唄30
BLACKLAGOONZERObulletMAX
BOOWY
CCエンジェル
CREANewクレアの秘宝伝
FAIRYTAIL2
GANTZ極THESURVIVALGAME
GI優駿倶楽部2
GI優駿倶楽部3
HEYエリートサラリーマン鏡
HEY鏡
KING黄門ちゃま
Lucky海物語
L主役は銭形4
OVERLORD絶対支配者光臨II
OVERSLOT「AINZOOALGOWN絶対支配者光臨」
PYRAMIDEYE
Reゼロから始める異世界生活
Reゼロから始める異世界生活ApexVacation
STEINSGATE
S戦姫絶唱シンフォギア
S牙狼黄金騎士
S稼動停止台
TIGERBUNNY
WakeUpGirlsSevenMemories
いろはに愛姫
うしおととら雷槍一閃
うまい棒
えとたま
かまいたちの夜
とある科学の超電磁砲
どんだけ
ひぐらしのなく頃に祭2
ひぐらしのなく頃に祭2カケラ遊び編
もっとクレアの秘宝伝女神の歌声と太陽の子供達
もっとチバリヨ25
もっとチバリヨ30
アイドルマスターミリオンライブ
アイムジャグラーEX
アイムジャグラーEXTP
アカメが斬る
アクエリオンALLSTARS
アナザーゴッドハーデス解き放たれし槍撃ver
アナターのオットはーです
アバサー
ウルトラマンタロウ暴君SPEC
ガールズケイリンGIフェアリーグランプリ
ガールズパンツァーGこれが私の戦車道です
ガールズパンツァー劇場版
キャッツアイ
キングハナハナ30
ギャグダー
ケロット4
ゲッターロボ
ゴッドイータージアニメーション
ゴブリンスレイヤー
サラリーマン金太郎MAX
サンダーVライトニング
シャア専用パチスロ逆襲の赤い彗星
シリウス
シンデレラブレイド4
スターパルサー
スナイパイ71
スロドル
スーパーハナハナ
スーパーハナハナ30
スーパーリオエース
スーパー海物語INJAPAN祭
ゼーガペイン2
タブータトゥー
ダンまち外伝ソードオラトリア
チバリヨ25
チバリヨ30
デビルメイクライ5
ドンちゃん2
ドンチャン2
ニューゲッターマウス
ニューハナハナゴールド
ニューパルサーDX3
ニューパルサーSPIII
ハイスクールD×D2ハーレム王に俺はなる
ハイスクールオブザデッドゴールド
ハイハイシオサイ
ハ

In [31]:
worksheet = workbook.worksheet('店舗名変換用テーブル完成版')
tenpo_name_convert_df = read_spreadsheet(workbook,worksheet)
tenpo_name_convert_df = tenpo_name_convert_df.replace({'スロパチID': {'': 99999}})
tenpo_name_convert_df = tenpo_name_convert_df.fillna({'スロパチID': 99999})
tenpo_name_convert_df['スロパチID'] = tenpo_name_convert_df['スロパチID'].astype(int)
tenpo_name_convert_df

,kintone_code,経度,緯度,DMM店舗名,都道府県,市区町村,住所,アナスロ店舗名,ホールナビ店舗名,URL,スロパチID,店舗名／法人名
1,P04961,139.378484,35.4501632,キコーナ海老名店,神奈川県,海老名市,神奈川県海老名市河原口4-5-6,キコーナ海老名店,キコーナ海老名店,https://p-town.dmm.com/shops/kanagawa/4961,506,キコーナ海老名店
2,P00651,139.80573,35.749407,ピーアーク北千住SSS,東京都,足立区,東京都足立区千住旭町3-5駅前会館ビル1Ｆ,ピーアーク北千住SSS,ピーアーク北千住SSS,https://p-town.dmm.com/shops/tokyo/651,513,ピーアーク北千住SSS
3,P01195,140.783334,41.7806327,KEIO湯川店,北海道,函館市,北海道函館市湯川町1-26-12,KEIO湯川店,KEIO湯川店,https://p-town.dmm.com/shops/hokkaido/1195,708,
4,P09568,133.904531,34.6159756,123藤田店,岡山県,岡山市南区,岡山県岡山市南区藤田673-22,123藤田店,123藤田店,https://p-town.dmm.com/shops/okayama/9568,771,123藤田店
5,P09572,133.937835,34.6277287,123浜野店,岡山県,岡山市南区,岡山県岡山市南区浜野2-10-48,123浜野店,123浜野店,https://p-town.dmm.com/shops/okayama/9572,772,
...,...,...,...,...,...,...,...,...,...,...,...,...
2406,,,,,,,,メガフェイス960飯塚,,,99999,MEGAFACE960飯塚
2407,,,,,,,,HYPER ARROW美原店,,,99999,HYPER ARROW 美原店
2408,,,,,,,,パールショップともえセンター南店,,,99999,パールショップともえセンター南店
2409,,,,,,,,123+N布施店,,,99999,123+N布施店


In [ ]:
sammy_sf_list = list(sammy_sf_report_df[sammy_sf_report_df['イベント日'] == '2023/03/31']['店舗名／法人名'].unique())
for x in sorted(sammy_sf_list):
    print(x)

In [7]:
sammy_sf_report_df = pd.read_csv(r'csv\sammy_sf_0331.csv')
#sammy_sf_report_df['shop_id'] = sammy_sf_report_df['shop_id'].astype(int)
sammy_sf_report_df = sammy_sf_report_df.rename(columns={'店舗ID':'スロパチID'})
sammy_sf_report_df = sammy_sf_report_df.fillna({'スロパチID': 99999})
sammy_sf_report_df['スロパチID'] = sammy_sf_report_df['スロパチID'].astype(int)
sammy_sf_report_df

# last_merged_df = pd.merge(sammy_sf_report_df,tenpo_name_convert_df,on='店舗名／法人名',how='left')
# anaslo_serched_tenpo_name_list = list(last_merged_df['アナスロ店舗名'].unique())
# anaslo_serched_tenpo_name_list.pop(6)
# anaslo_serched_tenpo_name_list = sorted(anaslo_serched_tenpo_name_list)
# #anaslo_serched_tenpo_name_list
# last_merged_df

,店舗名／法人名,都道府県,イベント日,提案区分,shop_id,スロパチID
0,123+N羽曳野店,大阪府,2023/03/31,サミーの日,7998.0,8940
1,123+N大阪本店,大阪府,2023/03/31,サミーの日,12564.0,8153
2,123+Ｎ松原店,大阪府,2023/03/31,サミーの日,7948.0,8826
3,123横浜西口店,神奈川県,2023/03/31,サミーの日,12985.0,970
4,123鶴橋店,大阪府,2023/03/31,サミーの日,7344.0,8395
...,...,...,...,...,...,...
63,新！ガーデン北与野,埼玉県,2023/03/31,サミーの日,3312.0,3588
64,仙川UNO,東京都,2023/03/31,サミーの日,885.0,5416
65,中山UNO,神奈川県,2023/03/31,サミーの日,4564.0,4157
66,中山ZoRoN,神奈川県,2023/03/31,サミーの日,12719.0,20615


In [ ]:
for tenpo_name in ['123門真店','123+N大阪本店','レッドロック学芸大学駅前店']:
    display(serched_df[serched_df['店舗名'] ==tenpo_name])
    serched_df[serched_df['店舗名'] ==tenpo_name].to_csv(fr'csv\{tenpo_name}_sammy_kisyu.csv',index=False)

In [38]:
date_list:list[str] = ['2023-05-02','2023-05-03']
#date_list:list[str] = ['03-30','03-31']
error_kisyu_name_list:list[str] = []
error_tenpo_name_list:list[str] = []
sammy_groupby_serched_df_list = []
for date_str in date_list:
    month_str = date_str.split('-')[0]
    #print(month_str)
    #anaslo_serched_tenpo_name_list:list[str] = list(set(kisyu_name_convert_df[f'{month_str}月_店舗名']))
    anaslo_serched_tenpo_name_list = tenpo_name_list
    print(len(anaslo_serched_tenpo_name_list))
    sorted(anaslo_serched_tenpo_name_list)
    #anaslo_serched_tenpo_name_list.remove('')
    #print(anaslo_serched_tenpo_name_list)
    csv_file_name  = f'csv\\{date_str}_scraping_anaslo.csv'
    target_date = csv_file_name.split('_')[0].split('\\')[-1]
    data_df =  pd.read_csv(csv_file_name)
    serched_df = pd.DataFrame(index=[], columns=list(data_df.columns))
    display(data_df)
    for serch_tenpo_name in list(data_df['店舗名'].unique()):
        #try:
        #print(serch_tenpo_name)
        extract_tenpo_data_df = data_df[data_df['店舗名'] == serch_tenpo_name]
        print(serch_tenpo_name,len(extract_tenpo_data_df))
        for kisyu_name in serch_kisyu_name_list :
            #print(kisyu_name)
            try:
                serch_kisyu_name_record = extract_tenpo_data_df[extract_tenpo_data_df['機種名'] == kisyu_name]
                if len(serch_kisyu_name_record) == 0:
                    error_kisyu_name_list.append(kisyu_name)# + f'_{date_str}'
                serched_df= pd.concat([serched_df, serch_kisyu_name_record], axis=0)
                #print('成功',kisyu_name)
            except:
                print('失敗',kisyu_name)
                error_kisyu_name_list.append(kisyu_name)
                continue
            #break
        # except:
        #     error_tenpo_name_list.append(serch_tenpo_name)
        #     continue
        #display(extract_tenpo_data_df)
        #break
        #break
    #serch_kisyu_name_list.remove('')
    serched_df['G数'] = serched_df['G数'].astype(int)
    groupby_serched_df = serched_df.groupby('店舗名').mean()#.sort_values('G数',ascending=False)#.reset_index()
    groupby_serched_df['サミー総台数'] = serched_df.groupby(['店舗名']).size()
    new_column_gamesuu_column:str = target_date + '_' + 'サミー全台平均G数'
    groupby_serched_df = groupby_serched_df.rename(columns={'G数':new_column_gamesuu_column})
    groupby_serched_df = groupby_serched_df[[ new_column_gamesuu_column,'サミー総台数']]#groupby_serched_df_list
    sammy_groupby_serched_df_list.append(groupby_serched_df)


93


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
0,332,123+N和歌山本店,モンキーターンIV,705,1666,-528.0,0,3,7.0,1/166.6,1/0.0,1/555.3,1/238.0,2023-05-02
1,333,123+N和歌山本店,モンキーターンIV,706,1052,-1003.0,0,2,2.0,1/263.0,1/0.0,1/526.0,1/526.0,2023-05-02
2,334,123+N和歌山本店,モンキーターンIV,707,998,-818.0,0,1,2.0,1/332.7,1/0.0,1/998.0,1/499.0,2023-05-02
3,368,123+N和歌山本店,ゼーガペイン2,708,3,-5.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-02
4,366,123+N和歌山本店,笑ゥせぇるすまん4,709,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28993,131,BEAM by HIKARI,沖ドキGOLD,1172,595,663.0,7,1,NaN,1/74.4,1/85.0,1/595.0,NaN,2023-05-02
28994,132,BEAM by HIKARI,沖ドキGOLD,1173,1734,142.0,13,2,NaN,1/115.6,1/133.4,1/867.0,NaN,2023-05-02
28995,133,BEAM by HIKARI,沖ドキGOLD,1175,3436,-4025.0,5,3,NaN,1/429.5,1/687.2,1/1145.3,NaN,2023-05-02
28996,134,BEAM by HIKARI,沖ドキGOLD,1176,507,1568.0,11,0,NaN,1/46.1,1/46.1,1/0.0,NaN,2023-05-02


123+N和歌山本店 396
123+N大阪本店 480
123+N布施店 661
123+N羽曳野店 443
123横浜西口店 332
123長吉店 240
123門真店 446
123難波店 376
123鶴橋店 391
123+N橿原店 423
BEAM HARA 311
Dステーション仙台東店 272
Dステーション上田店 255
GRAND G-ONE甲賀水口店 434
HYPER ARROW美原店 391
KING BOSS 1000 354
メガコンコルド1020刈谷知立店 426
メガコンコルド1111BLAZE店 482
メガコンコルド1280稲沢店 480
MGM小山本郷店 360
PIA京急川崎 298
PIA八王子 478
PIA川崎ダイス 306
PIA柏 スロット館 343
PIA横浜モアーズ 400
PIA町田新館 386
スーパーDステーション上里店 264
スーパーDステーション鴻巣店 360
WIN-WIN ONE 273
アスカ熊谷店 195
アスカ狭山店 240
アビバ湘南台店 501
アビバ関内 318
ジュラク王子店 381
エクスアリーナ柏 400
エクスアリーナ東京 480
キコーナ吹田店 320
キコーナ守口店 363
キコーナ小田原扇町店 240
キコーナ海老名店 433
キコーナ茨木店 280
グランキコーナ堺店 697
グランキコーナ相模原店 491
グランドオータ瀬谷店 406
サンパレス環七大杉店 287
ZAP PRESTIGE 152
ジアスセンター南 211
ジャムフレンドクラブ朝霞店 238
スロット123梶取店 235
テンガイ・与次郎店 400
パラッツォ三郷中央店 520
ビッグアップル.加古川 376
ピーアーク三田 183
ピーアークピーくんステーション 345
ピーアーク相模原 363
ピーアーク相模大野 450
ピーアーク竹ノ塚スタジオ 268
ピーアーク草加 260
プレイランド第一平和 214
マルハンいわき店 276
マルハン草薙アピア店 385
マルハン青梅新町店 530
ミリオン習志野店 160
メガコンコルド800春日井店+ 364
レッドロック学芸大学駅前店 143
UNO三ノ輪 250
上尾UNO 329
UNO上福岡 184
UNO中山 205
ZoRoN中山 264

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
0,332,123+N和歌山本店,モンキーターンIV,705,260,-135.0,0,1,1.0,1/130.0,1/0.0,1/260.0,1/260.0,2023-05-03
1,333,123+N和歌山本店,モンキーターンIV,706,1602,1108.0,0,1,12.0,1/123.2,1/0.0,1/1602.0,1/133.5,2023-05-03
2,334,123+N和歌山本店,モンキーターンIV,707,1654,-920.0,0,2,5.0,1/236.3,1/0.0,1/827.0,1/330.8,2023-05-03
3,368,123+N和歌山本店,ゼーガペイン2,708,1982,1225.0,6,5,6.0,1/116.6,1/330.3,1/396.4,1/330.3,2023-05-03
4,366,123+N和歌山本店,笑ゥせぇるすまん4,709,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28717,131,BEAM by HIKARI,沖ドキGOLD,1172,2214,-1890.0,6,3,NaN,1/246.0,1/369.0,1/738.0,NaN,2023-05-03
28718,132,BEAM by HIKARI,沖ドキGOLD,1173,3163,-2752.0,8,6,NaN,1/225.9,1/395.4,1/527.2,NaN,2023-05-03
28719,133,BEAM by HIKARI,沖ドキGOLD,1175,3198,-1312.0,16,4,NaN,1/159.9,1/199.9,1/799.5,NaN,2023-05-03
28720,134,BEAM by HIKARI,沖ドキGOLD,1176,4409,4833.0,47,20,NaN,1/65.8,1/93.8,1/220.4,NaN,2023-05-03


123+N和歌山本店 396
123+N大阪本店 480
123+N布施店 661
123+N羽曳野店 443
123横浜西口店 332
123長吉店 240
123門真店 446
123難波店 376
123鶴橋店 391
123+N橿原店 423
BEAM HARA 311
Dステーション上田店 255
GRAND G-ONE甲賀水口店 434
HYPER ARROW美原店 391
KING BOSS 1000 354
メガコンコルド1020刈谷知立店 426
メガコンコルド1111BLAZE店 482
メガコンコルド1280稲沢店 480
MGM小山本郷店 360
PIA京急川崎 298
PIA八王子 478
PIA川崎ダイス 306
PIA柏 スロット館 343
PIA横浜モアーズ 400
PIA町田新館 386
スーパーDステーション上里店 264
スーパーDステーション鴻巣店 360
WIN-WIN ONE 273
アスカ熊谷店 195
アスカ狭山店 240
アビバ湘南台店 501
アビバ関内 318
ジュラク王子店 381
エクスアリーナ柏 400
エクスアリーナ東京 480
キコーナ吹田店 320
キコーナ守口店 363
キコーナ小田原扇町店 240
キコーナ海老名店 433
キコーナ茨木店 280
グランキコーナ堺店 697
グランキコーナ相模原店 491
グランドオータ瀬谷店 406
サンパレス環七大杉店 287
ZAP PRESTIGE 152
ジアスセンター南 211
ジャムフレンドクラブ朝霞店 238
スロット123梶取店 235
テンガイ・与次郎店 400
パラッツォ三郷中央店 520
ビッグアップル.加古川 376
ピーアーク三田 183
ピーアークピーくんステーション 345
ピーアーク相模原 363
ピーアーク相模大野 450
ピーアーク竹ノ塚スタジオ 264
ピーアーク草加 260
プレイランド第一平和 214
マルハンいわき店 276
マルハン草薙アピア店 385
マルハン青梅新町店 530
ミリオン習志野店 160
メガコンコルド800春日井店+ 364
レッドロック学芸大学駅前店 143
UNO三ノ輪 250
上尾UNO 329
UNO上福岡 184
UNO中山 205
ZoRoN中山 264
UNO仙川 187
出玉王田無

In [40]:

error_kisyu_name_list:list[str] = []
error_tenpo_name_list:list[str] = []
tenpo_groupby_serched_df_list = []
for date_str in date_list:
    month_str = date_str.split('-')[0]
    print(month_str)
    #anaslo_serched_tenpo_name_list:list[str] = list(set(kisyu_name_convert_df[f'{month_str}月_店舗名']))
    print(len(anaslo_serched_tenpo_name_list))
    sorted(anaslo_serched_tenpo_name_list)
    #anaslo_serched_tenpo_name_list.remove('')
    print(anaslo_serched_tenpo_name_list)
    csv_file_name  = f'csv\\{date_str}_scraping_anaslo.csv'
    target_date = csv_file_name.split('_')[0].split('\\')[-1]
    data_df =  pd.read_csv(csv_file_name)
    serched_df = pd.DataFrame(index=[], columns=list(data_df.columns))
    for serch_tenpo_name in list(data_df['店舗名'].unique()):
        try:
            #print(serch_tenpo_name)
            # try:
            #     shop_id:int = int(slopachi_tenpo_name_and_shopid_df[slopachi_tenpo_name_and_shopid_df['店舗名／法人名'] == serch_tenpo_name].iloc[0,1])
            # except:
            #     print(serch_tenpo_name)
            #     continue
            try:
                #anaslo_tenpo_name = tenpo_name_convert_df[tenpo_name_convert_df['shop_id'] == shop_id].iloc[0,6]
                extract_tenpo_data_df = data_df[data_df['店舗名'] == serch_tenpo_name]
            except Exception as e :
                print(e)
                try:
                    extract_tenpo_data_df = data_df[data_df['店舗名'] == serch_tenpo_name]
                except Exception as e:
                    print('エラー',serch_tenpo_name,e)
                    error_tenpo_name_list.append(serch_tenpo_name)
                    continue
            serched_df= pd.concat([serched_df, extract_tenpo_data_df], axis=0)
        except Exception as e :
            error_tenpo_name_list.append(serch_tenpo_name)
            print(e)
            continue
        #display(extract_tenpo_data_df)
        #break
        #break
    #serch_kisyu_name_list.remove('')
    serched_df['G数'] = serched_df['G数'].astype(int)
    groupby_serched_df = serched_df.groupby('店舗名').mean()#.sort_values('G数',ascending=False)#.reset_index()
    groupby_serched_df['総台数'] = serched_df.groupby(['店舗名']).size()
    new_column_gamesuu_column:str = target_date + '_' + '全台平均G数'
    groupby_serched_df = groupby_serched_df.rename(columns={'G数':new_column_gamesuu_column})
    groupby_serched_df = groupby_serched_df[[ new_column_gamesuu_column,'総台数']]#groupby_serched_df_list
    tenpo_groupby_serched_df_list.append(groupby_serched_df)
    



2023
93
['123+N和歌山本店', '123+N大阪本店', '123+N布施店', '123+N羽曳野店', '123横浜西口店', '123長吉店', '123門真店', '123難波店', '123鶴橋店', '123+N橿原店', 'BEAM HARA', 'Dステーション仙台東店', 'Dステーション上田店', 'Dステーション新小岩店', 'EVO2', 'GRAND G-ONE甲賀水口店', 'HYPER ARROW美原店', 'KING BOSS 1000', 'MEGA BEAM ASAKURA 999', 'メガコンコルド1020刈谷知立店', 'メガコンコルド1111BLAZE店', 'メガコンコルド1280稲沢店', 'MGM小山本郷店', 'PIA京急川崎', 'PIA八王子', 'PIA川崎ダイス', 'PIA柏 スロット館', 'PIA横浜モアーズ', 'PIA町田新館', 'スーパーDステーション上里店', 'スーパーDステーション立川店', 'スーパーDステーション鴻巣店', 'WIN-WIN ONE', 'アスカ熊谷店', 'アスカ狭山店', 'アビバ湘南台店', 'アビバ関内', 'ジュラク王子店', 'エクスアリーナ柏', 'エクスアリーナ東京', 'オゼック国分寺北口', 'キコーナ吹田店', 'キコーナ守口店', 'キコーナ小田原扇町店', 'キコーナ海老名店', 'キコーナ茨木店', 'グランキコーナ堺店', 'グランキコーナ泉佐野店', 'グランキコーナ相模原店', 'グランドオータ瀬谷店', 'サンパレス環七大杉店', 'ZAP PRESTIGE', 'ジアスセンター南', 'ジャムフレンドクラブ朝霞店', 'スロット123梶取店', 'テンガイ・与次郎店', 'パラッツォ三郷中央店', 'ビッグアップル.加古川', 'ピーアーク三田', 'ピーアークピーくんステーション', 'ピーアーク相模原', 'ピーアーク相模大野', 'ピーアーク竹ノ塚スタジオ', 'ピーアーク草加', 'メガフェイス1180座間', 'プレイランド第一平和', 'マルハンいわき店', 'マルハン草薙アピア店', 'マルハン青梅新町店', 'ミリオン習志野店', 'メガコンコルド800春日井店+', 'レッドロック学芸大学駅前店',

In [41]:
pd.concat(tenpo_groupby_serched_df_list + sammy_groupby_serched_df_list,axis=1)

,2023-05-02_全台平均G数,総台数,2023-05-03_全台平均G数,総台数,2023-05-02_サミー全台平均G数,サミー総台数,2023-05-03_サミー全台平均G数,サミー総台数
店舗名,,,,,,,,
123+N和歌山本店,3073.441919,396,4057.446970,396.0,3888.712644,87,5758.459770,87.0
123+N大阪本店,6318.683333,480,5694.647917,480.0,6396.534653,101,6504.287129,101.0
123+N布施店,4193.432678,661,5187.069592,661.0,4306.559633,109,6258.550459,109.0
123+N橿原店,4442.789598,423,4778.683215,423.0,4855.806818,88,6051.670455,88.0
123+N羽曳野店,4625.636569,443,5015.934537,443.0,5276.870588,85,5922.882353,85.0
...,...,...,...,...,...,...,...,...
楽園川崎店,5818.522042,431,5213.767981,431.0,6310.289157,83,6543.518072,83.0
楽園沼津店,4115.917379,351,3429.310541,351.0,4400.722222,54,6387.518519,54.0
楽園浜松ザザシティ店,4871.515351,456,3437.651316,456.0,6198.901099,91,5628.054945,91.0


In [42]:
pd.concat(tenpo_groupby_serched_df_list + sammy_groupby_serched_df_list,axis=1).to_csv(r'csv\0503_0510出力1.csv')
#pd.concat(temo_df_list).to_csv('三か月分実施.csv')

In [ ]:
slopachi_tenpo_name_and_shopid_df =  pd.read_csv(r'csv\slopachi_tenpo_name_and_shopid.csv')
slopachi_tenpo_name_and_shopid_df =slopachi_tenpo_name_and_shopid_df[(~slopachi_tenpo_name_and_shopid_df['店舗ID'].isnull()) & (~slopachi_tenpo_name_and_shopid_df['都道府県'].isnull())]
slopachi_tenpo_name_and_shopid_df = slopachi_tenpo_name_and_shopid_df[['記事用店舗名','店舗ID','店舗名／法人名','都道府県']]
slopachi_tenpo_name_and_shopid_df['店舗ID'] = slopachi_tenpo_name_and_shopid_df['店舗ID'].astype(int)
slopachi_tenpo_name_and_shopid_df.drop_duplicates(subset='店舗名／法人名', keep='last', inplace=True)
slopachi_tenpo_name_and_shopid_df

In [50]:
serched_df['日付'] = f"2023-{date}"
serched_df.to_csv(rf'csv\2023-{date}_scraping_anaslo.csv',index=False)

In [66]:
data_df.to_csv(r'csv\2023-03-02_anaslo copy.csv',index=False)

In [ ]:
csv_file_name  = f'csv\\2023-03-03_anaslo copy.csv'
target_date = csv_file_name.split('_')[0].split('\\')[-1]
data_df =  pd.read_csv(csv_file_name)
data_df



====== WebDriver manager ======
Current google-chrome version is 112.0.5615
Get LATEST chromedriver version for 112.0.5615 google-chrome
Driver [C:\Users\slotd\.wdm\drivers\chromedriver\win32\112.0.5615.49\chromedriver.exe] found in cache
C:\Users\slotd\AppData\Local\Temp/ipykernel_50400/1895281330.py:51: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


NameError: name 'prefecture_name_list' is not defined

In [6]:
sf_past_all_record_df = pd.read_csv(r'csv\sf_past_all_record.csv')
sf_past_all_record_df = sf_past_all_record_df[~sf_past_all_record_df['店舗ID'].isnull()]
sf_past_all_record_df = sf_past_all_record_df[~sf_past_all_record_df.duplicated(subset=['店舗名／法人名'])]
sf_past_all_record_df = sf_past_all_record_df[['店舗名／法人名','店舗ID']]
sf_past_all_record_df['店舗ID'] = sf_past_all_record_df['店舗ID'].astype(int)
sf_past_all_record_df.reset_index(drop=True,inplace=True)
sf_past_all_record_df = sf_past_all_record_df.rename(columns={'店舗ID':'スロパチID'})
sf_past_all_record_df



,店舗名／法人名,スロパチID
0,オゼック羽村,5488
1,ガーデン松戸金ヶ作,3281
2,パーラートモエ,4851
3,スロットスタジアムフォーション,4898
4,TAKARAJIMA荻島店,3723
...,...,...
1517,LOVE,7456
1518,スロット123梶取店,7370
1519,アミューズ岩出,851
1520,メトロヒルズ,7371


In [12]:
parlar_name_parlar_url_dict_df = pd.read_csv(r'csv\tenpo_name_tenpo_url_dict.csv', header=None, names=['店舗名', 'url'])
parlar_name_parlar_url_dict_df
parlar_name_parlar_url_dict = d
ict(zip(parlar_name_parlar_url_dict_df['店舗名'],parlar_name_parlar_url_dict_df['url'] ))
#parlar_name_parlar_url_dict

In [25]:
import csv
dct = slopachi_and_anaslo_tenpo_name_dict
with open(r'csv\slopachi_and_anaslo_tenpo_name_dict.csv', 'w') as f:  
    writer = csv.writer(f)
    for k, v in dct.items():
       writer.writerow([k, v])

In [8]:
for i,tenpo_name in enumerate(tenpo_name_list):
    anaslo_parlar_name_url = parlar_name_parlar_url_dict[tenpo_name]
    anaslo_url_parlarname =urllib.parse.unquote((anaslo_parlar_name_url.split('/')[-2])).replace('-データ一覧','')
    print(i,anaslo_url_parlarname)

NameError: name 'tenpo_name_list' is not defined

In [43]:
list(set(anaslo_scraping_error_tenpo_name_list))

['Dステーション新小岩店',
 'メガフェイス1180座間',
 'グランキコーナ泉佐野店',
 'Dステーション仙台東店',
 'オゼック国分寺北口',
 'スーパーDステーション立川店',
 'MEGA BEAM ASAKURA 999',
 'EVO2']

In [45]:
hand_computed_tenpo_name_list = '''レッドロック学芸大学駅前店
ZAP PRESTIGE
第一プラザ西浦和店'''.split('\n')

In [47]:
for tenpo_name in hand_computed_tenpo_name_list:
    display(serched_df[serched_df['店舗名'] ==tenpo_name])
    serched_df[serched_df['店舗名'] ==tenpo_name].to_csv(fr'csv\{tenpo_name}_2023-05-03.csv',index=False)

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
22747,131,レッドロック学芸大学駅前店,犬夜叉,1,0,0.0,0,0,NaN,1/0.0,1/0.0,1/0.0,NaN,2023-05-03
22748,133,レッドロック学芸大学駅前店,バーサスリヴァイズ,2,4418,1448.0,21,12,NaN,1/133.9,1/210.4,1/368.2,NaN,2023-05-03
22749,134,レッドロック学芸大学駅前店,新ハナビ,3,2882,151.0,11,6,NaN,1/169.5,1/262.0,1/480.3,NaN,2023-05-03
22750,129,レッドロック学芸大学駅前店,ハードボイルド,5,3251,-207.0,10,5,NaN,1/216.7,1/325.1,1/650.2,NaN,2023-05-03
22751,130,レッドロック学芸大学駅前店,ハードボイルド,6,1507,-1256.0,2,0,NaN,1/753.5,1/753.5,1/0.0,NaN,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22885,64,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,213,3064,-207.0,12,7,NaN,1/161.3,1/255.3,1/437.7,NaN,2023-05-03
22886,65,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,215,1852,-207.0,13,5,NaN,1/102.9,1/142.5,1/370.4,NaN,2023-05-03
22887,66,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,216,2657,262.0,15,5,NaN,1/132.8,1/177.1,1/531.4,NaN,2023-05-03
22888,67,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,217,1850,-2130.0,5,0,NaN,1/370.0,1/370.0,1/0.0,NaN,2023-05-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
16821,38,ZAP PRESTIGE,ハッピージャグラーVIII,385,1617,-706.0,3,5,NaN,1/202.1,1/539.0,1/323.4,NaN,2023-05-03
16822,39,ZAP PRESTIGE,ハッピージャグラーVIII,386,630,-355.0,1,1,NaN,1/315.0,1/630.0,1/630.0,NaN,2023-05-03
16823,40,ZAP PRESTIGE,ハッピージャグラーVIII,387,3484,-1352.0,6,12,NaN,1/193.6,1/580.7,1/290.3,NaN,2023-05-03
16824,41,ZAP PRESTIGE,ハッピージャグラーVIII,388,767,-324.0,2,1,NaN,1/255.7,1/383.5,1/767.0,NaN,2023-05-03
16825,42,ZAP PRESTIGE,ハッピージャグラーVIII,389,2438,-179.0,10,3,NaN,1/187.5,1/243.8,1/812.7,NaN,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16968,67,ZAP PRESTIGE,沖ドキGOLD,536,3882,-1464.0,19,7,NaN,1/149.3,1/204.3,1/554.6,NaN,2023-05-03
16969,68,ZAP PRESTIGE,沖ドキGOLD,537,4084,591.0,26,16,NaN,1/97.2,1/157.1,1/255.2,NaN,2023-05-03
16970,69,ZAP PRESTIGE,沖ドキGOLD,538,5607,-1924.0,27,15,NaN,1/133.5,1/207.7,1/373.8,NaN,2023-05-03
16971,70,ZAP PRESTIGE,沖ドキGOLD,539,4539,6205.0,55,19,NaN,1/61.3,1/82.5,1/238.9,NaN,2023-05-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
25239,103,第一プラザ西浦和店,ハッピージャグラーVIII,449,2103,-1309.0,4,3,0.0,1/300.4,1/525.8,1/701.0,1/0.0,2023-05-03
25240,104,第一プラザ西浦和店,ハッピージャグラーVIII,450,8520,1652.0,38,28,0.0,1/129.1,1/224.2,1/304.3,1/0.0,2023-05-03
25241,105,第一プラザ西浦和店,ハッピージャグラーVIII,451,2900,-1993.0,5,4,0.0,1/322.2,1/580.0,1/725.0,1/0.0,2023-05-03
25242,63,第一プラザ西浦和店,ファンキージャグラー2,452,5349,209.0,24,10,0.0,1/157.3,1/222.9,1/534.9,1/0.0,2023-05-03
25243,64,第一プラザ西浦和店,ファンキージャグラー2,453,3769,-1952.0,8,8,0.0,1/235.6,1/471.1,1/471.1,1/0.0,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25386,26,第一プラザ西浦和店,HEYエリートサラリーマン鏡,596,3908,335.0,16,7,0.0,1/169.9,1/244.2,1/558.3,1/0.0,2023-05-03
25387,27,第一プラザ西浦和店,HEYエリートサラリーマン鏡,597,5915,-2497.0,23,11,0.0,1/174.0,1/257.2,1/537.7,1/0.0,2023-05-03
25388,28,第一プラザ西浦和店,HEYエリートサラリーマン鏡,598,2268,-1367.0,12,2,0.0,1/162.0,1/189.0,1/1134.0,1/0.0,2023-05-03
25389,29,第一プラザ西浦和店,HEYエリートサラリーマン鏡,599,2700,-1660.0,11,3,0.0,1/192.9,1/245.5,1/900.0,1/0.0,2023-05-03


In [51]:
hand_comupted_sammy_serched_df = pd.DataFrame(index=[], columns=list(data_df.columns))
for tenpo_name in hand_computed_tenpo_name_list:
    
    display(serched_df[serched_df['店舗名'] ==tenpo_name])
    extract_tenpo_data_df = concat_data_df[concat_data_df['店舗名'] ==tenpo_name]
    for kisyu_name in serch_kisyu_name_list :
        #print(kisyu_name)
        try:
            serch_kisyu_name_record = extract_tenpo_data_df[extract_tenpo_data_df['機種名'] == kisyu_name]
            if len(serch_kisyu_name_record) == 0:
                error_kisyu_name_list.append(kisyu_name)# + f'_{date_str}'
            hand_comupted_sammy_serched_df = pd.concat([hand_comupted_sammy_serched_df, serch_kisyu_name_record], axis=0)
            
            #print('成功',kisyu_name)
        except:
            pass
    hand_comupted_sammy_serched_df.to_csv(fr'csv\hand_computed_sammy_2023-05-03.csv',index=False)

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
22747,131,レッドロック学芸大学駅前店,犬夜叉,1,0,0.0,0,0,NaN,1/0.0,1/0.0,1/0.0,NaN,2023-05-03
22748,133,レッドロック学芸大学駅前店,バーサスリヴァイズ,2,4418,1448.0,21,12,NaN,1/133.9,1/210.4,1/368.2,NaN,2023-05-03
22749,134,レッドロック学芸大学駅前店,新ハナビ,3,2882,151.0,11,6,NaN,1/169.5,1/262.0,1/480.3,NaN,2023-05-03
22750,129,レッドロック学芸大学駅前店,ハードボイルド,5,3251,-207.0,10,5,NaN,1/216.7,1/325.1,1/650.2,NaN,2023-05-03
22751,130,レッドロック学芸大学駅前店,ハードボイルド,6,1507,-1256.0,2,0,NaN,1/753.5,1/753.5,1/0.0,NaN,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22885,64,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,213,3064,-207.0,12,7,NaN,1/161.3,1/255.3,1/437.7,NaN,2023-05-03
22886,65,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,215,1852,-207.0,13,5,NaN,1/102.9,1/142.5,1/370.4,NaN,2023-05-03
22887,66,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,216,2657,262.0,15,5,NaN,1/132.8,1/177.1,1/531.4,NaN,2023-05-03
22888,67,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,217,1850,-2130.0,5,0,NaN,1/370.0,1/370.0,1/0.0,NaN,2023-05-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
16821,38,ZAP PRESTIGE,ハッピージャグラーVIII,385,1617,-706.0,3,5,NaN,1/202.1,1/539.0,1/323.4,NaN,2023-05-03
16822,39,ZAP PRESTIGE,ハッピージャグラーVIII,386,630,-355.0,1,1,NaN,1/315.0,1/630.0,1/630.0,NaN,2023-05-03
16823,40,ZAP PRESTIGE,ハッピージャグラーVIII,387,3484,-1352.0,6,12,NaN,1/193.6,1/580.7,1/290.3,NaN,2023-05-03
16824,41,ZAP PRESTIGE,ハッピージャグラーVIII,388,767,-324.0,2,1,NaN,1/255.7,1/383.5,1/767.0,NaN,2023-05-03
16825,42,ZAP PRESTIGE,ハッピージャグラーVIII,389,2438,-179.0,10,3,NaN,1/187.5,1/243.8,1/812.7,NaN,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16968,67,ZAP PRESTIGE,沖ドキGOLD,536,3882,-1464.0,19,7,NaN,1/149.3,1/204.3,1/554.6,NaN,2023-05-03
16969,68,ZAP PRESTIGE,沖ドキGOLD,537,4084,591.0,26,16,NaN,1/97.2,1/157.1,1/255.2,NaN,2023-05-03
16970,69,ZAP PRESTIGE,沖ドキGOLD,538,5607,-1924.0,27,15,NaN,1/133.5,1/207.7,1/373.8,NaN,2023-05-03
16971,70,ZAP PRESTIGE,沖ドキGOLD,539,4539,6205.0,55,19,NaN,1/61.3,1/82.5,1/238.9,NaN,2023-05-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
25239,103,第一プラザ西浦和店,ハッピージャグラーVIII,449,2103,-1309.0,4,3,0.0,1/300.4,1/525.8,1/701.0,1/0.0,2023-05-03
25240,104,第一プラザ西浦和店,ハッピージャグラーVIII,450,8520,1652.0,38,28,0.0,1/129.1,1/224.2,1/304.3,1/0.0,2023-05-03
25241,105,第一プラザ西浦和店,ハッピージャグラーVIII,451,2900,-1993.0,5,4,0.0,1/322.2,1/580.0,1/725.0,1/0.0,2023-05-03
25242,63,第一プラザ西浦和店,ファンキージャグラー2,452,5349,209.0,24,10,0.0,1/157.3,1/222.9,1/534.9,1/0.0,2023-05-03
25243,64,第一プラザ西浦和店,ファンキージャグラー2,453,3769,-1952.0,8,8,0.0,1/235.6,1/471.1,1/471.1,1/0.0,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25386,26,第一プラザ西浦和店,HEYエリートサラリーマン鏡,596,3908,335.0,16,7,0.0,1/169.9,1/244.2,1/558.3,1/0.0,2023-05-03
25387,27,第一プラザ西浦和店,HEYエリートサラリーマン鏡,597,5915,-2497.0,23,11,0.0,1/174.0,1/257.2,1/537.7,1/0.0,2023-05-03
25388,28,第一プラザ西浦和店,HEYエリートサラリーマン鏡,598,2268,-1367.0,12,2,0.0,1/162.0,1/189.0,1/1134.0,1/0.0,2023-05-03
25389,29,第一プラザ西浦和店,HEYエリートサラリーマン鏡,599,2700,-1660.0,11,3,0.0,1/192.9,1/245.5,1/900.0,1/0.0,2023-05-03
